In [1]:
#imports personal details to connect to db
import config
#import our function files
import query_helper
import payscale_uni_webscrape


import json
import requests
from bs4 import BeautifulSoup
api_key = config.API_key
import time 
import mysql.connector
from mysql.connector import errorcode
import pandas as pd

# Below 2 cells creates database

In [ ]:
# db_name = 'colleges'
# ## Connect to DB server on AWS
# ## connecting to the database using 'connect()' method
# ## it takes 3 required parameters 'host', 'user', 'passwd'
# cnx = mysql.connector.connect(
#     host = config.host,
#     user = config.user,
#     passwd = config.password,
#     database = "colleges"       #can say what database we expect everything loaded into
# )
# cursor = cnx.cursor()

In [ ]:
# def create_database(cursor, database):
#     try:
#         cursor.execute(
#             "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
#     except mysql.connector.Error as err:
#         print("Failed creating database: {}".format(err))
#         exit(1)

# try:
#     cursor.execute("USE {}".format(db_name))
# except mysql.connector.Error as err:
#     print("Database {} does not exists.".format(db_name))
#     if err.errno == errorcode.ER_BAD_DB_ERROR:
#         create_database(cursor, db_name)
#         print("Database {} created successfully.".format(db_name))
#         cnx.database = db_name
#     else:
#         print(err)
#         exit(1)

In [ ]:
#write a query entry to creat a table
payscale_colleges = """
CREATE TABLE payscale_colleges (
      uni varchar(100) PRIMARY KEY,
      uni_type varchar(100),
      early_car_pay int,
      mid_car_pay int,
      high_meaning float,
      stem_deg float
    );
"""

In [ ]:
query_helper.create_table(payscale_colleges)

In [2]:
#Below code gets info for page then commits to db, repeats process on all pages
#if we have duplicates they will be skipped on db commit process

web_page_iterator = 1
while web_page_iterator <64:
    #scrape web infor for 1 page
    list_of_dict = payscale_uni_webscrape.get_pages(web_page_iterator)

    #put infor into tuples and commit to db
    query_helper.tuple_payscale_uni(list_of_dict)
    
    #iterate and loop
    web_page_iterator +=1
    
